In [294]:
import re
import requests
import json
import subprocess
import os
import tempfile
import pandas as pd
import gseapy as gp
from openai import OpenAI

This functions helps if protein accession is given returns fetches fafsa sequence for the protein

In [295]:
UNIPROT_API_URL = "https://rest.uniprot.org/uniprotkb/"

def fetch_fasta_sequence(accession: str) -> str:
    accession = accession.strip().replace(".", "")
    url = f"{UNIPROT_API_URL}{accession}.fasta"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        return f"Error fetching FASTA: {str(e)}"

**perform_blast_search** :function retrieves a FASTA sequence for a given accession ID and performs a BLAST search using it. It returns the original sequence and formatted BLAST results, handling temporary file creation and cleanup.


In [296]:

def run_blast(fasta_file: str) -> str:
    fasta_file_docker = fasta_file.replace("\\", "/")
    cmd = [
        "docker", "run", "--rm",
        "-v", f"{fasta_file_docker}:/input.faa",
        "ncbi/blast",
        "blastp", "-query", "/input.faa", "-db", "swissprot", "-outfmt", "6", "-remote"
    ]
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        return result.stdout or "No hits found."
    except subprocess.CalledProcessError as e:
        return f"BLAST error: {e.stderr}"

def perform_blast_search(accession: str) -> dict:
    fasta_sequence = fetch_fasta_sequence(accession)
    if "Error" in fasta_sequence:
        return {"error": fasta_sequence}
    with tempfile.NamedTemporaryFile(delete=False, mode='w', suffix='.faa') as f:
        f.write(fasta_sequence)
        temp_path = f.name
    try:
        blast_results = run_blast(temp_path)
    finally:
        os.remove(temp_path)
    """
    return {
        "fasta_sequence": fasta_sequence,
        "blast_results": format_blast_results(blast_results)
    }
    """
    return blast_results

**get_uniprot_info** : function fetches protein information from the UniProt API using a given accession ID. It returns key details like protein name, organism, and sequence, or an error message if the request fails.


In [297]:
def get_uniprot_info(accession: str) -> dict:
    url = f"{UNIPROT_API_URL}{accession}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return {
            "accession": data.get("primaryAccession"),
            "name": data.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value"),
            "organism": data.get("organism", {}).get("scientificName"),
            "sequence": data.get("sequence", {}).get("value")
        }
    except requests.exceptions.RequestException as e:
        return {"error": f"API failed: {str(e)}"}

**perform_gsea_enrichr** : function performs Gene Set Enrichment Analysis using the Enrichr API for a given list of genes and a specified gene set (defaulting to "KEGG_2016"). It returns the enrichment results as a dictionary or an error message if the analysis fails.

In [298]:
def perform_gsea_enrichr(genes, gene_sets="KEGG_2016"):
    try:
        enr = gp.enrichr(gene_list=genes, gene_sets=gene_sets, outdir=None, cutoff=0.05)
        results = enr.results
        return {
            "input_genes": genes,
            "gene_sets": gene_sets,
            "results": results.to_dict(orient="records")
        }
    except Exception as e:
        return {"error": str(e)}

**run_tomtom** : function executes the TOMTOM motif comparison tool inside a Docker container using specified query and database motif files. It returns the contents of the TOMTOM result file or an error message if the execution fails.

In [299]:
def run_tomtom(motif_file: str, database: str) -> str:
    motif_abs = os.path.abspath(motif_file)
    db_abs = os.path.abspath(database)
    output_dir = tempfile.mkdtemp()
    cmd = [
        "docker", "run", "--rm",
        "-v", f"{motif_abs}:/input/query.meme",
        "-v", f"{db_abs}:/input/db.meme",
        "-v", f"{output_dir}:/output",
        "memesuite/memesuite",
        "tomtom", "/input/query.meme", "/input/db.meme", "-oc", "/output"
    ]
    try:
        subprocess.run(cmd, check=True)
        result_file = os.path.join(output_dir, "tomtom.tsv")
        if os.path.exists(result_file):
            with open(result_file) as f:
                return f.read()
        else:
            return "TOMTOM completed, but no result file was found."
    except subprocess.CalledProcessError as e:
        return f"TOMTOM error: {e.stderr}"
    except Exception as ex:
        return f"Unexpected error: {str(ex)}"

**fetch_go_term_info** : function retrieves the name and aspect (e.g., Biological Process, Molecular Function) of a given Gene Ontology (GO) term from the QuickGO API. If the request fails, it returns default values `"N/A"`.


In [300]:
def fetch_go_term_info(go_id: str) -> tuple:
    try:
        url = f"https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/{go_id}"
        headers = {"Accept": "application/json"}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
        result = data.get("results", [{}])[0]
        return (
            result.get("name", "N/A"),
            result.get("aspect", "N/A")
        )
    except Exception:
        return ("N/A", "N/A")

**get_go_annotations** : function retrieves Gene Ontology (GO) annotations for a UniProt ID from the QuickGO API, filtered by specified aspects. It returns GO IDs, term names, aspects, and evidence codes, or an error message if the request fails.


In [301]:
def get_go_annotations(uniprot_id: str, aspects=["biological_process", "molecular_function", "cellular_component"]) -> dict:
    base_url = "https://www.ebi.ac.uk/QuickGO/services/annotation/search"
    headers = {"Accept": "application/json"}
    params = {
        "geneProductId": f"UniProtKB:{uniprot_id}",
        "aspect": ",".join(aspects),
        "limit": 50
    }
    try:
        response = requests.get(base_url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        annotations = []
        for item in data.get("results", []):
            go_id = item.get("goId")
            evidence = item.get("evidenceCode", "N/A")
            term_name, aspect = fetch_go_term_info(go_id)
            annotations.append({
                "GO_ID": go_id,
                "GO_Term": term_name,
                "Aspect": aspect,
                "Evidence": evidence
            })
        return {"UniProtID": uniprot_id, "GO_annotations": annotations}
    except requests.exceptions.RequestException as e:
        return {"error": f"QuickGO request failed: {str(e)}"}

**get_protein_interactions** : function queries the STRING database API to retrieve protein-protein interaction data for a list of gene IDs in a specified species (default is human, species ID 9606). It returns the interaction network in JSON format or an error message if the request fails.


In [302]:
def get_protein_interactions(gene_ids, species=9606):
    url = "https://string-db.org/api/json/network"
    params = {
        "identifiers": "%0A".join(gene_ids),
        "species": species
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": f"STRING DB request failed: {str(e)}"}

**get_alphafold_model** : function retrieves a predicted protein structure from AlphaFold for a given UniProt ID. It returns key details such as the protein description, a sample of confidence scores, and the model download URL, or an error message if retrieval fails.

In [ ]:
def get_alphafold_model(uniprot_id: str) -> dict:
    try:
        api_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"

        print(f"Fetching protein info for UniProt ID: {uniprot_id}")
        response = requests.get(api_url)

        if response.status_code == 200:
            data = response.json()
            return json.dumps(data, indent=2)
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        return {"error": f"AlphaFold request failed: {str(e)}"}
    except Exception as ex:
        return {"error": f"Unexpected error: {str(ex)}"}

**predict_ptm_musite** : function queries the MusiteDeep API to predict post-translational modification (PTM) sites for a given protein sequence using selected models. It returns prediction results or an error message if the request fails.

In [304]:
def predict_ptm_musite(sequence: str, selected_models: str = "Phosphoserine_Phosphothreonine;O-linked_glycosylation") -> dict:
    url = f"https://api.musite.net/musitedeep/{selected_models}/{sequence}"
    try:
        response = requests.get(url, timeout=15)
        if response.ok:
            jData = response.json()
            if "Error" in jData:
                return {"error": jData["Error"]}
            return {
                "PTM_models_used": selected_models.split(";"),
                "sequence_length": len(sequence),
                "ptm_predictions": jData
            }
        return {"error": f"Musite API failed: {response.status_code} - {response.text}"}
    except Exception as e:
        return {"error": f"Request exception: {str(e)}"}

**run_mmseqs_pairwise** : function performs a pairwise sequence alignment using MMseqs2 in a Docker container on a given FASTA file. It returns a formatted result table with query, target, e-value, and bitscore, or an error message if the process fails.


In [305]:
def run_mmseqs_pairwise(query_fasta: str, output_dir: str = "./mmseqs_output") -> str:
    query_abs = os.path.abspath(query_fasta)
    output_dir_abs = os.path.abspath(output_dir)
    os.makedirs(output_dir_abs, exist_ok=True)
    cmd = [
        "docker", "run", "--rm",
        "-v", f"{query_abs}:/data/query.fasta",
        "-v", f"{output_dir_abs}:/data/output",
        "soedinglab/mmseqs2",
        "easy-search", "/data/query.fasta", "/data/query.fasta", "/data/output/result.m8", "/data/output",
        "--format-output", "query,target,evalue,bits"
    ]
    try:
        subprocess.run(cmd, check=True)
        result_file = os.path.join(output_dir_abs, "result.m8")
        if os.path.exists(result_file):
            with open(result_file) as f:
                lines = f.read().strip().split("\n")
            headers = ["Query", "Target", "E-value", "Bitscore"]
            rows = [line.split("\t") for line in lines]
            col_widths = [
                max(len(str(row[col_idx])) for row in [headers] + rows)
                for col_idx in range(len(headers))
            ]
            table = [
                " | ".join(f"{header:<{width}}" for header, width in zip(headers, col_widths)),
                "-" * (sum(col_widths) + 3 * (len(headers) - 1))
            ]
            for row in rows:
                table.append(" | ".join(f"{item:<{width}}" for item, width in zip(row, col_widths)))
            return "\n".join(table)
        return "MMseqs2 completed but no result file was found."
    except subprocess.CalledProcessError as e:
        return f"MMseqs2 error: {e.stderr}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

**get_binding_sites** : function retrieves binding site annotations for a protein using its UniProt accession ID. It returns a list of binding site positions and descriptions or an error message if the retrieval fails.

In [306]:
def get_binding_sites(accession: str) -> str:
    try:
        response = requests.get(f"{UNIPROT_API_URL}{accession}.json", timeout=10)
        features = response.json().get("features", [])
        bindings = [f for f in features if f.get("type") == "Binding site"]
        if not bindings:
            return "No binding sites found."
        return "\n".join([
            f"Position: {b['location']['start']}, Description: {b.get('description', 'N/A')}"
            for b in bindings
        ])
    except Exception as e:
        return f"Error: {e}"

**get_function** : function fetches the functional description of a protein from UniProt using its accession ID. It extracts and returns the first available function comment or an appropriate message if not found or if an error occurs.

In [307]:
def get_function(accession: str) -> str:
    try:
        response = requests.get(f"{UNIPROT_API_URL}{accession}.json", timeout=10)
        data = response.json()
        for comment in data.get("comments", []):
            if comment.get("commentType") == "FUNCTION":
                return comment.get("texts", [{}])[0].get("value", "Function not available.")
        return "Function not found."
    except Exception as e:
        return f"Error: {e}"

**get_localization** : function retrieves subcellular localization information for a protein using its UniProt accession ID. It returns a list of locations where the protein is found or an error message if the information is unavailable.

In [308]:
def get_localization(accession: str) -> str:
    try:
        response = requests.get(f"{UNIPROT_API_URL}{accession}.json", timeout=10)
        comments = response.json().get("comments", [])
        localizations = []
        for c in comments:
            if c.get("commentType") == "SUBCELLULAR LOCATION":
                for loc in c.get("subcellularLocations", []):
                    localizations.append(loc.get("location", {}).get("value", ""))
        return "\n".join(localizations) if localizations else "No subcellular localization found."
    except Exception as e:
        return f"Error: {e}"

In [309]:
def extract_accession(line: str) -> list:
    """
    Extract UniProt accession numbers or protein IDs from a line of text.
    Supports both Swiss-Prot (e.g., P12345) and TrEMBL (e.g., Q8N158) formats.
    """
    pattern = r'\b([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9]{5})\b'
    return re.findall(pattern, line)

***Driver_Code***

In [310]:
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
MODEL = "llama3.2:1b"
function_map = {
    "get_blast": perform_blast_search,
    "get_tomtom": run_tomtom,
    "get_gsea": perform_gsea_enrichr,
    "get_go_annotation": get_go_annotations,
    "get_protein_interaction": get_protein_interactions,
    "get_mmseq": run_mmseqs_pairwise,
    "get_predict_ptm": predict_ptm_musite,
    "get_function": get_function,
    "get_binding_sites": get_binding_sites,
    "get_subcellular_location": get_localization,
    "get_alphafold": get_alphafold_model,
}

# === Use Ollama to decide which function to call ===

def ask_llm_for_function_call(user_input):
    prompt = f"""
You are a bioinformatics assistant. Your job is to map a user's query to the correct function name from a fixed list.

You must return **only the function name**, nothing else. No parentheses, no explanation.

Choose from these valid function names:

- get_blast
- get_tomtom
- get_gsea
- get_go_annotation
- get_protein_interaction
- get_mmseq
- get_predict_ptm
- get_function
- get_binding_sites
- get_subcellular_location
- get_alphafold_model

Here are example queries for each function:

---

🔹 get_blast – Run a BLAST search for a protein  
Examples:
- Run a BLAST search for P05067  
- BLAST this sequence for similarity  
- Perform BLAST search on protein ID P04637  
- Search homologous proteins using BLAST  
- BLAST protein P12345 against database  
- Compare P05067 with other sequences using BLAST  
- Use BLAST to analyze this protein  
- Find similar sequences to this protein  
- Run blastp for this UniProt ID  
- Get sequence similarity using BLAST

---

🔹 get_alphafold – Get predicted 3D protein structure from AlphaFold  
Examples:
- Get AlphaFold model for Q5VSL9  
- Show predicted structure from AlphaFold  
- Fetch AlphaFold results for this UniProt ID  
- Retrieve structural model from AlphaFold  
- View protein 3D structure from AlphaFold  
- Predicted model from AlphaFold database  
- Use AlphaFold to get structure for P04637  
- Access AlphaFold protein prediction  
- Show AlphaFold result for this protein  
- Download AlphaFold structure

---

🔹 get_tomtom – Compare motifs using TOMTOM  
Examples:
- Compare motifs using TOMTOM  
- Use TOMTOM to find similar motifs  
- Run TOMTOM against JASPAR database  
- Perform motif comparison using TOMTOM  
- Identify motif matches using TOMTOM  
- Run TOMTOM search with my query.meme file  
- Compare this motif to the database with TOMTOM  
- Motif search via TOMTOM  
- Find transcription factor motif similarities using TOMTOM  
- Analyze motif similarity with TOMTOM

---

🔹 get_gsea – Perform GSEA for gene list  
Examples:
- Run GSEA on TP53, BRCA1, and BRCA2  
- Perform gene set enrichment for my gene list  
- Do KEGG enrichment analysis  
- Analyze pathway enrichment for TP53-related genes  
- Perform enrichment analysis using KEGG pathways  
- Run GSEA for cancer-related genes  
- Conduct gene set analysis  
- Execute enrichment analysis for these genes  
- Analyze functional pathways using GSEA  
- Check enrichment scores for selected genes

---

🔹 get_go_annotation – Get Gene Ontology annotation  
Examples:
- Get GO annotation for P04637  
- What GO terms are linked to P05067?  
- Show biological processes for this protein  
- Fetch GO terms for P04637  
- Display cellular component terms for this ID  
- Give me molecular function GO terms for P04637  
- Retrieve GO data for this protein  
- List gene ontology entries for this UniProt ID  
- GO classification for this gene  
- Show ontology categories for this protein

---

🔹 get_protein_interaction – Get interaction data  
Examples:
- Get protein interaction for TP53 and BRCA1  
- Show interaction network for TP53  
- Which proteins interact with BRCA1?  
- Find interaction partners of TP53  
- Fetch STRING interactions for these genes  
- Retrieve protein–protein interaction data  
- Show BRCA1 interacting partners  
- Get list of interactors for TP53  
- Load protein interaction graph  
- Analyze molecular interaction for these genes

---

🔹 get_mmseq – Run MMseqs2  
Examples:
- Run MMseqs for P05067  
- Search MMseqs2 for this UniProt ID  
- Use MMseq to find similar sequences  
- MMseq protein sequence alignment for P05067  
- Perform MMseqs2 search  
- Search database using MMseqs  
- Find fast protein alignments with MMseqs  
- Identify protein families via MMseqs  
- Search MMseqs with protein FASTA  
- Align sequences using MMseqs2

---

🔹 get_predict_ptm – Predict post-translational modifications  
Examples:
- Predict PTMs for P04637  
- Show post-translational modifications for this protein  
- Are there phosphorylation sites on P04637?  
- Find ubiquitination sites on this protein  
- PTM prediction for this UniProt ID  
- Analyze potential modifications in P04637  
- Predict glycosylation sites  
- What PTMs are predicted for this sequence?  
- Check modification sites on this protein  
- Run PTM analysis on P04637

---

🔹 get_function – Get protein function description  
Examples:
- What does P04637 do?  
- Describe the function of this protein  
- What is the biological role of P05067?  
- Get protein function for P04637  
- Show functional description of this gene  
- What is the function of this UniProt ID?  
- Explain what P04637 does in the cell  
- Provide function summary for this gene  
- Molecular role of P04637  
- What activity does this protein perform?

---

🔹 get_binding_sites – Get binding site locations  
Examples:
- Where are the binding sites on P04637?  
- Locate active sites in this protein  
- Show ligand binding domains for P04637  
- What regions bind DNA or ligands?  
- Find binding regions in this protein  
- Display catalytic sites for this gene  
- Which amino acids are involved in binding?  
- Highlight metal binding sites on P04637  
- Get interaction domains in protein structure  
- Are there substrate binding residues in this sequence?

---

🔹 get_subcellular_location – Get cellular location info  
Examples:
- Where is P04637 located in the cell?  
- Show subcellular localization of this protein  
- Is this protein nuclear or cytoplasmic?  
- Where does P04637 function in the cell?  
- In which organelle is this gene expressed?  
- What is the localization of this UniProt ID?  
- Determine if protein is membrane-bound  
- Is this protein mitochondrial?  
- Cellular component of P04637?  
- Where is this protein found inside the cell?

---

Now return just the function name (e.g., `get_function`) that best matches this query:
"{user_input}"
"""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# === Process single user command ===

def process_command(user_input):
    
    try:
        func_name = ask_llm_for_function_call(user_input)
        print(f"> Function to call: {func_name}")  # Debugging output
        func = function_map.get(func_name)
        print("function being called : "+ str(func))
        
        if perform_blast_search == func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            return json.dumps(perform_blast_search(accession), indent=2)
        elif predict_ptm_musite == func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            fasta = fetch_fasta_sequence(accession)
            if "Error" in fasta:
                return json.dumps({"error": fasta}, indent=2)
            seq = ''.join([line.strip() for line in fasta.splitlines() if not line.startswith(">")])
            return json.dumps(predict_ptm_musite(seq), indent=2)
        elif get_go_annotations == func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            return json.dumps(get_go_annotations(accession), indent=2)
        elif get_protein_interactions == func:
            print(1)
            gene_ids = [g.strip() for g in re.findall(r"[A-Z0-9]+", user_input)]
            print(gene_ids)
            return json.dumps(get_protein_interactions(gene_ids), indent=2)
        elif perform_gsea_enrichr == func:
            print(1)
            gene_list = extract_accession(user_input)
            return json.dumps(perform_gsea_enrichr(gene_list), indent=2)
        elif get_function == func:
            print(1)
            accession  = str(extract_accession(user_input)[0])
            return get_function(accession)

        elif get_binding_sites == func:
            print(1)
            accession  = str(extract_accession(user_input)[0])
            return get_binding_sites(accession)

        elif get_localization ==func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            return get_localization(accession)
        elif run_mmseqs_pairwise == func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            fasta_sequence = fetch_fasta_sequence(accession)
            print("Fafsa seq:"+ fasta_sequence)
            if "Error" in fasta_sequence:
                return {"error": fasta_sequence}
                
            with tempfile.NamedTemporaryFile(mode='w', suffix='.fasta', delete=False) as f:
                f.write(fasta_sequence)
                temp_fasta = f.name
                
            try:
                print(1000)
                result = run_mmseqs_pairwise(temp_fasta)
                return result
            finally:
                os.remove(temp_fasta)
        elif get_alphafold_model ==func:
            print(1)
            accession = str(extract_accession(user_input)[0])
            return get_alphafold_model(accession)
        else:
            print(2)
            accession = extract_accession(user_input)
            return json.dumps(get_uniprot_info(accession), indent=2)
    except Exception as e:
        return f"Error interpreting your request: {str(e)}"
    
# === Read and process queries from file ===

def process_queries_from_file(file_path: str):
    with open(file_path) as f:
        queries = [line.strip() for line in f if line.strip()]
    for i, query in enumerate(queries, 1):
        print(f"\nQUERY {i}: {query}")
        result = process_command(query)
        print(f"RESPONSE:\n{result}\n{'='*50}")

# === Entry point ===

if __name__ == "__main__":
    process_queries_from_file("queries.txt")


QUERY 1: Fetch AlphaFold results for this Q5VSL9
> Function to call: get_alphafold
function being called : <function get_alphafold_model at 0x000001C07D951E40>
1
Fetching protein info for UniProt ID: Q5VSL9
RESPONSE:
[
  {
    "entryId": "AF-Q5VSL9-F1",
    "gene": "STRIP1",
    "sequenceChecksum": "5F9BA1D4C7DE6925",
    "sequenceVersionDate": "2004-12-07",
    "uniprotAccession": "Q5VSL9",
    "uniprotId": "STRP1_HUMAN",
    "uniprotDescription": "Striatin-interacting protein 1",
    "taxId": 9606,
    "organismScientificName": "Homo sapiens",
    "uniprotStart": 1,
    "uniprotEnd": 837,
    "uniprotSequence": "MEPAVGGPGPLIVNNKQPQPPPPPPPAAAQPPPGAPRAAAGLLPGGKAREFNRNQRKDSEGYSESPDLEFEYADTDKWAAELSELYSYTEGPEFLMNRKCFEEDFRIHVTDKKWTELDTNQHRTHAMRLLDGLEVTAREKRLKVARAILYVAQGTFGECSSEAEVQSWMRYNIFLLLEVGTFNALVELLNMEIDNSAACSSAVRKPAISLADSTDLRVLLNIMYLIVETVHQECEGDKAEWRTMRQTFRAELGSPLYNNEPFAIMLFGMVTKFCSGHAPHFPMKKVLLLLWKTVLCTLGGFEELQSMKAEKRSILGLPPLPEDSIKVIRNMRAASPPASASDLIEQQQKRGRREHKALIKQDNLDAFNERDPYKADD